## 1. For 5 seed yolov7 tiny weights, calculating mean +- std

In [3]:
import subprocess
import re
import pandas as pd
import numpy as np
import sys
import os

# --- Configuration ---
seeds = [0, 1, 2, 3, 4]
data_yaml = "data/DUP_bal_TrainVal.yaml"  # UPDATE THIS if your yaml name is different
img_size = 320
conf_thres = 0.001
iou_thres = 0.65  # Standardized threshold
device = "0"      # GPU device

# UPDATE THIS PATH PATTERN based on where your weights are stored relative to the script
# Based on your previous message: runs/train/2_yolov7_tiny_seed_0/weights/last.pt
def get_weight_path(seed):
    return f"runs/train/2_yolov7_tiny_seed_{seed}/weights/last.pt"

# --- Storage for results ---
all_results = []

print(f"Starting YOLOv7-tiny evaluation for {len(seeds)} seeds...")
print("-" * 60)

for seed in seeds:
    weight_path = get_weight_path(seed)
    
    # Check if weight file exists to avoid crashes
    if not os.path.exists(weight_path):
        print(f"Error: Weight file not found at {weight_path}")
        continue

    print(f"Processing Seed {seed}: {weight_path}")
    
    # Construct the YOLOv7 test command
    command = [
        "python", "test.py",
        "--weights", weight_path,
        "--data", data_yaml,
        "--img-size", str(img_size),
        "--conf-thres", str(conf_thres),
        "--iou-thres", str(iou_thres),
        "--task", "test",
        "--device", device,
        "--name", f"eval_yolov7_seed_{seed}"
    ]
    
    try:
        # Run the command and capture output
        result = subprocess.run(
            command, 
            capture_output=True, 
            text=True, 
            encoding='utf-8'
        )
        
        # Parse the output
        output_text = result.stdout + result.stderr
        
        # Regex to find the result lines (Class, Images, Instances, P, R, mAP50, mAP50-95)
        # Matches lines like: "   all   536   1044   0.948   0.938   0.977   0.864"
        pattern = r'\s+(all|U|D|P)\s+(\d+)\s+(\d+)\s+([0-9.]+)\s+([0-9.]+)\s+([0-9.]+)\s+([0-9.]+)'
        
        matches = re.findall(pattern, output_text)
        
        if not matches:
            print(f"Warning: No metrics found for seed {seed}.")
            # Print error if command failed
            if result.returncode != 0:
                print(f"Command Error:\n{result.stderr}")
        
        for match in matches:
            cls_name, imgs, insts, p, r, map50, map95 = match
            all_results.append({
                "Seed": seed,
                "Class": cls_name,
                "Images": int(imgs),
                "Instances": int(insts),
                "Precision": float(p),
                "Recall": float(r),
                "mAP50": float(map50),
                "mAP50-95": float(map95)
            })
            
        print(f"Seed {seed} completed.")

    except Exception as e:
        print(f"An exception occurred processing seed {seed}: {e}")

print("-" * 60)
print("Evaluation Complete. Processing Data...")

# --- Data Processing ---
if not all_results:
    print("No results were captured. Please check paths and commands.")
    sys.exit(1)

df = pd.DataFrame(all_results)

# 1. Save Raw Output to CSV
csv_filename = "yolov7_evaluation_results.csv"
df.to_csv(csv_filename, index=False)
print(f"\nRaw results saved to {csv_filename}")

# 2. Calculate Mean and Std
stats = df.groupby('Class')[['mAP50', 'mAP50-95']].agg(['mean', 'std'])

# Map class codes to full names
class_names = {'all': 'All', 'U': 'Ascending', 'D': 'Descending', 'P': 'Passing'}

print("\n" + "="*80)
print(f"{'YOLOv7-tiny FINAL RESULTS (Mean ± Std)':^80}")
print("="*80)
print(f"{'Class':<15} | {'mAP@0.5':<25} | {'mAP@0.5:0.95':<25}")
print("-" * 80)

for code, name in class_names.items():
    if code in stats.index:
        # Get stats
        m50_mean = stats.loc[code, ('mAP50', 'mean')]
        m50_std = stats.loc[code, ('mAP50', 'std')]
        
        m95_mean = stats.loc[code, ('mAP50-95', 'mean')]
        m95_std = stats.loc[code, ('mAP50-95', 'std')]
        
        # Format strings (e.g., "97.60 ± 0.30 %")
        res_50 = f"{m50_mean*100:.2f} ± {m50_std*100:.2f} %"
        res_95 = f"{m95_mean*100:.2f} ± {m95_std*100:.2f} %"
        
        print(f"{name:<15} | {res_50:<25} | {res_95:<25}")

print("="*80)

Starting YOLOv7-tiny evaluation for 5 seeds...
------------------------------------------------------------
Processing Seed 0: runs/train/2_yolov7_tiny_seed_0/weights/last.pt
Seed 0 completed.
Processing Seed 1: runs/train/2_yolov7_tiny_seed_1/weights/last.pt
Seed 1 completed.
Processing Seed 2: runs/train/2_yolov7_tiny_seed_2/weights/last.pt
Seed 2 completed.
Processing Seed 3: runs/train/2_yolov7_tiny_seed_3/weights/last.pt
Seed 3 completed.
Processing Seed 4: runs/train/2_yolov7_tiny_seed_4/weights/last.pt
Seed 4 completed.
------------------------------------------------------------
Evaluation Complete. Processing Data...

Raw results saved to yolov7_evaluation_results.csv

                     YOLOv7-tiny FINAL RESULTS (Mean ± Std)                     
Class           | mAP@0.5                   | mAP@0.5:0.95             
--------------------------------------------------------------------------------
All             | 97.58 ± 0.29 %            | 80.72 ± 1.11 %           
Ascendi

In [2]:
pwd

'/mnt/Documents/Dad/github/DUP/yolov7_dup'